In [ ]:
import os
import time
import pickle

import cv2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from scipy import ndimage as nd
from skimage import transform
from tqdm.notebook import tqdm

from astellaslib import *

#import isx

## Gather data from spreadsheet

In [ ]:
log_fn = '/home/mmiller/Downloads/Experimental Log - Sheet1.csv'
log_df = pd.read_csv(log_fn)
log_df.head()

In [ ]:
#behavior_dir = '/home/mmiller/Documents/data/astellas/fmr1ko_linear_social/April2020_FMR1_LinearSocial/'
#behavior_dir = fix
com_list = [i for i in os.listdir(behavior_dir) if os.path.splitext(i)[1] == '.csv']
#print(com_list)


In [ ]:
print(com_list)

In [ ]:
print(log_df.subject_ID.values)

In [ ]:
log_df.condition.unique()

In [ ]:
vid_df = log_df[['subject_ID', 'genotype','data_dir_behavior','condition', 'behav_data_basename', 'novel_side']].loc[
    (~log_df.behav_data_basename.isna()) & (log_df.condition.isin(['linear_social'])) & (log_df.virus.isin(['naive']))].copy()

In [ ]:
vid_df

### Examine frames from videos for spot checking etc :

In [ ]:
subj_ids = vid_df.subject_ID.unique()
print(vid_df.behav_data_basename.loc[vid_df.subject_ID==subj_ids[0]].values[0]+'_COM.mp4')
print(subj_ids)

In [ ]:
# examine behavior zones on example video:
ds_fact = 1
x_threshs = [95,230]
subj_id = 0
framei = int(60*20*7)


In [ ]:
print(vid_df.data_dir_behavior.loc[vid_df.subject_ID==subj_ids[subj_id]].values[0])
print(fix_data_path(vid_df.data_dir_behavior.loc[vid_df.subject_ID==subj_ids[subj_id]].values[0]))

In [ ]:
# examine behavior zones on example video:

#vid_fn = '/home/mmiller/Documents/data/astellas/fmr1ko_linear_social/April2020_FMR1_LinearSocial/behavior_2020-04-01-14-34-36_video_COM.mp4'
behavior_dir = fix_data_path(vid_df.data_dir_behavior.loc[vid_df.subject_ID==subj_ids[subj_id]].values[0])
vid_fn = behavior_dir + vid_df.behav_data_basename.loc[vid_df.subject_ID==subj_ids[subj_id]].values[0]+'_COM.mp4'
print(subj_ids[subj_id])

vid = cv2.VideoCapture(vid_fn)
frame_height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
vid.set(1, framei)
theframe = vid.read()[1]
#grayscale = transform.downscale_local_mean( cv2.cvtColor(theframe,cv2.COLOR_BGR2GRAY).astype('uint8'), (ds_fact,ds_fact)).astype('uint8')
vid.release()

f,ax = plt.subplots(1,1,figsize=(20,5))
ax.imshow(theframe,cmap='gray')
ax.vlines(x_threshs, ymin=0, ymax=frame_height-1, color='salmon')
plt.show()

#### Optionally create / populate dictionary for target sides:

In [ ]:
#test_side=dict()
test_side[subj_ids[subj_id]] = 'l'
print(test_side)

## Load existing dict of target sides if available:

In [ ]:
test_side_fn = 'target_sides_baclofen.pkl'
test_side = pickle.load(open(test_side_fn, 'rb'))

In [ ]:
test_side

In [ ]:
subj_ids = vid_df.subject_ID.values
print(subj_ids)

In [ ]:
vid_df['novel_side'] = ''

In [ ]:
for i in test_side:
    print(i)
    vid_df['novel_side'] = vid_df['subject_ID'].map(test_side[i])

#vid_df['novel_side'] = vid_df['subject_ID'].map(test_side)

In [ ]:
vid_df

### Create dict for occupancy times:

In [ ]:
the_cond = 'linear_social'

## count l/r occupancy for each epoch for each subject:
epoch_times = [0, int(5*60*20), int(15*60*20)]
buffer_len = 30*20

occupancy_dict = dict()


for subj in np.unique(subj_ids):
    print(subj)
    com_fn = behavior_dir + vid_df.behav_data_basename.loc[(vid_df.subject_ID==subj) & (vid_df.condition == the_cond)].values[0]+'_COM.csv'
    #print(com_fn)
    the_com = np.loadtxt(com_fn, delimiter=',')
    
    the_epochs = list(np.copy(epoch_times))
    the_epochs.append(len(the_com))
    epoch_segs = [[i+buffer_len, j-buffer_len] for i,j in zip(the_epochs[0:-1], the_epochs[1:])]
    
    occupancy_times = []
    for seg_i, seg in enumerate(epoch_segs):
        print(seg)
        left_time = int(sum(the_com[seg[0]:seg[1], 1] < x_threshs[0]))
        right_time = int(sum(the_com[seg[0]:seg[1], 1] > x_threshs[1]))
        #out_times[subj_i, seg_i, :] = [left_time, right_time]
        
        if seg_i == 0:
            occupancy_times.append([left_time, right_time])
            print('Habituation scores',left_time,right_time)        
        
        if seg_i == 1: 
            if vid_df.novel_side.loc[(vid_df.subject_ID == subj) & (vid_df.condition == the_cond)].values == 'l': # correctly order times to map social target position to the right ([object, pref])
                occupancy_times.append([right_time, left_time])
                print('remap scores:', right_time,left_time)
            else:
                occupancy_times.append([left_time, right_time])      
                print('unmapped scores',left_time,right_time)
        if seg_i == 2: 
            if vid_df.novel_side.loc[(vid_df.subject_ID == subj) & (vid_df.condition == the_cond)].values == 'l': # correctly order times to map novel target position to the right ([pref, novel])
                occupancy_times.append([left_time, right_time])
                print('unmapped scores:', left_time,right_time)
            else:
                occupancy_times.append([right_time, left_time])      
                print('remapped scores',right_time, left_time)

    occupancy_dict[subj] = occupancy_times
    print('\n')
    
print(occupancy_dict)

In [ ]:
occupancy_dict

### populate dataframe with occupancy scores:

In [ ]:

habituation_times = {i:j[0] for i,j in zip(occupancy_dict.keys(), occupancy_dict.values())}
print(habituation_times)


pref_times = {i:occupancy_dict[i][1] for i in occupancy_dict}
print(pref_times)

novel_times = {i:occupancy_dict[i][2] for i in occupancy_dict}
print(novel_times)



In [ ]:
vid_df['habituation_counts'] = ''
vid_df['pref_counts'] = ''
vid_df['novel_counts'] = ''



In [ ]:
vid_df['habituation_counts'].loc[(vid_df.condition == the_cond)] = vid_df['subject_ID'].map(habituation_times)
vid_df['pref_counts'].loc[(vid_df.condition == the_cond)] = vid_df['subject_ID'].map(pref_times)
vid_df['novel_counts'].loc[(vid_df.condition == the_cond)] = vid_df['subject_ID'].map(novel_times)

In [ ]:
vid_df

In [ ]:
vid_df.loc[(vid_df.condition == the_cond) & (vid_df.genotype == 'FMR1CTRL')]

In [ ]:
vid_df.loc[(vid_df.condition == 'linear_social') & (vid_df.genotype == 'FMR1CTRL')]

In [ ]:
print(vid_df.genotype.unique())

### Plotting:

In [ ]:
the_cond = 'linear_social'

## plot / test occupancy:

gtypes = vid_df.genotype.unique()
cmap = plt.get_cmap('tab10')
#cdict = dict(zip(gtypes, [cmap(i) for i in np.linspace(0, 1 , len(gtypes))]))
cdict = dict(zip(gtypes, [cmap(i) for i in range(len(gtypes))][::-1]))

f,ax = plt.subplots(1, 2, figsize = (20,5), sharey=True)

for the_gtype,a in zip(gtypes,ax):
    the_subjs = vid_df.subject_ID.loc[(vid_df.genotype == the_gtype) & (vid_df.condition == the_cond)]

    for i in vid_df.habituation_counts.loc[(vid_df.subject_ID.isin(the_subjs)) & (vid_df.condition == the_cond) ]:
        a.plot([1, 1.1], [j/20 for j in i], 'o-', color = cdict[the_gtype])
    for i in vid_df.pref_counts.loc[(vid_df.subject_ID.isin(the_subjs)) & (vid_df.condition == the_cond)]:
        a.plot([1.2, 1.3], [j/20 for j in i], 'o-', color = cdict[the_gtype])
    for i in vid_df.novel_counts.loc[(vid_df.subject_ID.isin(the_subjs)) & (vid_df.condition == the_cond)]:
        a.plot([1.4, 1.5], [j/20 for j in i], 'o-', color = cdict[the_gtype])
        

for a, gtype in zip(ax, gtypes):
    a.set_title(gtype)
    a.set_xticks([1, 1.1, 1.2, 1.3, 1.4, 1.5])
    a.set_xticklabels(['left', 'right', 'object', 'conspecific 1', 'conspecific 1', 'conspecific 2'], rotation = 345, fontsize=12)
    a.set_ylim(0,500)
    a.set_yticks([0,250,500])
    a.set_ylabel('occupancy time (s)', fontsize=12)
    twoaxis(a)
plt.show()


